In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [5]:
train_data_dir = '/scratch/hle/caltech_101/train'
categories = os.listdir(train_data_dir)
categories_one_hot = LabelBinarizer().fit_transform(categories)
image_input_shape = (224, 224, 3,)
batch_size = 32

In [6]:
print(len(categories_one_hot))
print(len(categories))

101
101


In [7]:
mean = [103.939, 116.779, 123.68]
def prep_image(im):    
    # Convert to BGR
    im = im[:,:, ::-1]
    im[..., 0] -= mean[0]
    im[..., 1] -= mean[1]
    im[..., 2] -= mean[2]
    return im
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=prep_image,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=image_input_shape[:-1],
        batch_size=batch_size,
        class_mode='categorical')

Found 4310 images belonging to 101 classes.


In [8]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y = tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')




def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))
    conv3_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 128, 256], mean=0, stddev=0.08))
    conv4_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 256, 512], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
  
    # 5, 6
    conv3 = tf.nn.conv2d(conv2_bn, conv3_filter, strides=[1,1,1,1], padding='SAME')
    conv3 = tf.nn.relu(conv3)
    conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  
    conv3_bn = tf.layers.batch_normalization(conv3_pool)
    
    # 7, 8
    conv4 = tf.nn.conv2d(conv3_bn, conv4_filter, strides=[1,1,1,1], padding='SAME')
    conv4 = tf.nn.relu(conv4)
    conv4_pool = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv4_bn = tf.layers.batch_normalization(conv4_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv4_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    # 12
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=tf.nn.relu)
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)    
    
    # 13
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=1024, activation_fn=tf.nn.relu)
    full4 = tf.nn.dropout(full4, keep_prob)
    full4 = tf.layers.batch_normalization(full4)        
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full4, num_outputs=10, activation_fn=None)
    return out

In [9]:
epochs = 100
batch_size = 128
keep_probability = 0.7
learning_rate = 0.1

In [11]:
logits = conv_net(x, keep_prob)

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer, 
                feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: keep_probability
})
    
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    loss = sess.run(cost, 
                    feed_dict={
                        x: feature_batch,
                        y: label_batch,
                        keep_prob: 1.
                    })
#     valid_acc = sess.run(accuracy, 
#                          feed_dict={
#                              x: valid_features,
#                              y: valid_labels,
#                              keep_prob: 1.
#                          })
    
#     print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, valid_acc))
    print('Loss: {:>10.4f}'.format(loss))

In [ ]:
image_batches = np.array(list(chunks(train_imgs, batch_size)))
label_batches = np.array(list(chunks(train_classes, batch_size)))

trained_model_path = '/scratch/hle/data/'
saver = tf.train.Saver(max_to_keep=20)

with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    i = 0
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        for j in range(1,5):
            batch_features = image_batches[i]
            batch_labels = one_hot_encoding(label_batches[i])
            train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
            i+=1      
            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, j), end='')
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
        if (epoch % 5 == 0):
            save_path = saver.save(sess, trained_model_path + ('%s/'% 'caltech101_models') +\
                                   "caltech_classifier_model_%s.ckpt" %  str(epoch+1))
            print("Model saved in file: %s" % save_path)